In [ ]:
import os
import numpy as np

In [ ]:
# # Download YOLOv7 repository and install requirements
# !git clone https://github.com/WongKinYiu/yolov7
# %cd yolov7
# %pip install -r requirements.txt

In [ ]:
# !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

# Yolo augment data internally so no need to augment it

In [1]:
!python ./train.py --batch 8 --epochs 2 --data ./data/data.yaml --weights './runs/train/yolov7/weights/best.pt' --name yolov7 --img 640 640 --cf cfg/training/yolov7.yaml --hyp ./data/hyp.scratch.p5.yaml --noautoanchor --freeze 2 --notest
## Training with diffrent hyperparameters
# !python ./train.py --batch 8 --epochs 4 --data ./data/data.yaml --weights './runs/train/yolov7/weights/best.pt' --name yolov7 --img 640 640 --cf cfg/training/yolov7.yaml --hyp ./data/hyp.scratch.p6.yaml --notest --noautoanchor --freeze 2


2024-08-18 00:02:04.964506: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-18 00:02:09.823328: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-18 00:02:17.408981: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-18 00:02:17.910944: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-18 00:02:20.097037: W tensorflow/compiler/tf2

In [18]:
!python ./test.py --conf-thres 0.2 --batch-size 4 --data ./data/data.yaml --weights './runs/train/yolov7/weights/best.pt' --name yolov7 --img-size 640

Namespace(weights=['./runs/train/yolov7/weights/best.pt'], data='./data/data.yaml', batch_size=4, img_size=640, conf_thres=0.2, iou_thres=0.65, task='val', device='', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='runs/test', name='yolov7', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 f18e862 torch 2.3.1+cu121 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/home/charbel/.local/lib/python3.11/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36524924 parameters, 6194944 gradients, 103.3 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced

# Detection over a folder of images

In [17]:
# Run evaluation
!python ./detect.py --weights ./runs/train/yolov7/weights/best.pt --conf 0.1 --source ../yolov7_data/test/images


Namespace(weights=['./runs/train/yolov7/weights/best.pt'], source='../yolov7_data/test/images', img_size=640, conf_thres=0.1, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 f18e862 torch 2.3.1+cu121 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/home/charbel/.local/lib/python3.11/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36524924 parameters, 6194944 gradients, 103.3 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

4 forklifts, 4 rack

# Export the model to onnx and torchscript

In [14]:
!python ./export.py --weights ./runs/train/yolov74/weights/best.pt --img-size 640 640 --include-nms --end2end

Namespace(weights='./runs/train/yolov74/weights/best.pt', img_size=[640, 640], batch_size=1, dynamic=False, dynamic_batch=False, grid=False, end2end=True, max_wh=None, topk_all=100, iou_thres=0.45, conf_thres=0.25, device='cpu', simplify=False, include_nms=True, fp16=False, int8=False)
YOLOR 🚀 f18e862 torch 2.3.1+cu121 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
/home/charbel/.local/lib/python3.11/site-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36524924 parameters, 6194944 gradients, 103.3 GFLOPS

Starting TorchScript export with torch 2.3.1+cu121...
TorchScript export success, saved as ./runs/train/yolov74/weights/best.torchscript.pt
scikit-learn version 1

In [ ]:
# display inference on ALL test images

import glob
from IPython.display import Image, display

i = 0
limit = 10000  # max images to print
for imageName in glob.glob("./runs/detect/exp/*.png"):  # assuming JPG
    if i < limit:
        display(Image(filename=imageName))
        print("\n")
    i = i + 1

In [16]:
!tensorboard --logdir ./runs/train/yolov7

2024-08-17 19:11:05.999834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 19:11:06.026634: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 19:11:06.034237: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-17 19:11:06.052606: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-17 19:11:07.254011: W tensorflow/compiler/tf2